In [1]:
import pandas as pd
import re
from cleantext import clean

In [2]:
dataset = pd.read_csv('../Data/all_data.csv')

In [3]:
len(dataset)

1999516

In [4]:
dataset

,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,...,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count
0,1083994,He got his money... now he lies in wait till a...,train,2017-03-06 15:21:53.675241+00,21,NaN,317120,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,67
1,650904,Mad dog will surely put the liberals in mental...,train,2016-12-02 16:44:21.329535+00,21,NaN,154086,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
2,5902188,And Trump continues his lifelong cowardice by ...,train,2017-09-05 19:05:32.341360+00,55,NaN,374342,approved,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,63
3,7084460,"""while arresting a man for resisting arrest"".\...",test,2016-11-01 16:53:33.561631+00,13,NaN,149218,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
4,5410943,Tucker and Paul are both total bad ass mofo's.,train,2017-06-14 05:08:21.997315+00,21,NaN,344096,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999511,1018736,Another man shamming article. If white men did...,train,2017-02-20 07:20:49.964620+00,54,NaN,169202,approved,0,0,...,0.8,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,10,10
1999512,340016,"""no matter what is put in front of you regardi...",train,2016-06-06 06:43:04.780968+00,21,339965.0,137961,approved,0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,10,10
1999513,919629,The Democrat party aided and abetted by it's M...,train,2017-01-30 02:44:29.168863+00,54,NaN,164845,rejected,0,1,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,11,10
1999514,5165492,I just don't find her a very good representati...,train,2017-04-22 18:42:02.442987+00,54,NaN,328877,approved,1,0,...,0.0,0.0,0.0,0.0,0.003717,0.0,0.0,0.00000,269,10


# Targets Labels

In [5]:
targets = ['toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat']

In [6]:
dataset[['comment_text', 'split', 'toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat']]

,comment_text,split,toxicity,obscene,sexual_explicit,identity_attack,insult,threat
0,He got his money... now he lies in wait till a...,train,0.373134,0.089552,0.014925,0.000000,0.343284,0.014925
1,Mad dog will surely put the liberals in mental...,train,0.605263,0.065789,0.013158,0.092105,0.565789,0.065789
2,And Trump continues his lifelong cowardice by ...,train,0.666667,0.031746,0.000000,0.047619,0.666667,0.000000
3,"""while arresting a man for resisting arrest"".\...",test,0.815789,0.552632,0.592105,0.000000,0.684211,0.105263
4,Tucker and Paul are both total bad ass mofo's.,train,0.550000,0.337500,0.275000,0.037500,0.487500,0.000000
...,...,...,...,...,...,...,...,...
1999511,Another man shamming article. If white men did...,train,0.400000,0.000000,0.000000,0.200000,0.300000,0.000000
1999512,"""no matter what is put in front of you regardi...",train,0.400000,0.000000,0.000000,0.400000,0.000000,0.000000
1999513,The Democrat party aided and abetted by it's M...,train,0.400000,0.200000,0.100000,0.300000,0.300000,0.000000
1999514,I just don't find her a very good representati...,train,0.400000,0.000000,0.000000,0.200000,0.100000,0.000000


In [7]:
dataset['comment_text'].iloc[1999511]

'Another man shamming article. If white men did do the work there would be no Canada today.\n\n*\n\n\n\n\n\n\n\n\nhttps://www.youtube.com/watch?v=4ZiBKdjJKgk'

In [8]:
dataset = dataset.copy()
dataset['comment_text'] = dataset['comment_text'].replace('', None) # .replace('', np.nan).dropna(axis=1, how='all')
dataset.dropna(subset=['comment_text'], inplace=True)

In [9]:
len(dataset)

1999512

In [10]:
https_url_pattern = re.compile(r'https?://\S+|www\.\S+') 
remove_tags = re.compile('<.*?>')

def remove_urls_and_htmlTags(text, replacement_text=''):
    # Define a regex pattern to match URLs
    text_without_urls = https_url_pattern.sub(replacement_text, text)
    result = remove_emojis(text_without_urls)
    return re.sub(remove_tags, replacement_text, result)

def remove_emojis (text):
    text = text.replace('"', '')
    if text != '':        
        text = clean(text,
                    # no_emoji=True, 
                    extra_spaces=True,
                    lowercase=False) 
    cleaned_text = re.sub(r'&#[0-9]+;', '', text)
    cleaned_text = text.replace('\n', ' ')
    cleaned_text = re.sub(r'@\w+', '[USER]', text)
    return cleaned_text

In [11]:
dataset['comment_text'] = dataset['comment_text'].apply(lambda x : remove_urls_and_htmlTags(x))

In [12]:
dataset[dataset['comment_text'] == '']

,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,...,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count
11024,663684,,train,2016-12-05 16:25:56.911397+00,21,NaN,154637,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4
11047,6034809,,train,2017-09-27 15:32:44.934458+00,53,6034548.0,379540,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4
11654,6149937,,train,2017-10-15 10:23:55.022876+00,22,NaN,387012,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10
11984,662993,,train,2016-12-05 14:11:04.854874+00,53,660988.0,154194,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,6
12862,5222859,,train,2017-05-06 12:51:49.062901+00,54,5222824.0,332418,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980711,5304064,,train,2017-05-25 19:55:34.126117+00,54,NaN,260072,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10
1980989,372424,,train,2016-07-02 01:49:30.607623+00,13,371920.0,140427,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10
1981077,477370,,train,2016-09-16 18:53:08.754386+00,21,476962.0,146089,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10
1981093,7067664,,test,2017-09-06 21:47:17.726127+00,55,NaN,374605,rejected,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10


In [13]:
dataset = dataset[dataset['comment_text'].str.strip().astype(bool)]

In [14]:
dataset.loc[:, 'comment_text'] = dataset['comment_text'].str.replace('\n', '', regex=False)

In [15]:
dataset[['toxicity',  'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat']] = dataset[['toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat']].map(lambda x: 1 if x >= .5 else 0)

In [16]:
dataset.to_csv('../Data/all_data_cleaned.csv')